# Downsampling Hamiltonian Dynamics

Consider a Hamiltonian system $ \mathcal{H}: \mathbb{R}^{2N} \rightarrow \mathbb{R} $. I wish to construct a map $ x: \mathbb{R}^{2N} \rightarrow \mathbb{R}^{2M} $ with $ M < N $. Intuitively, I want to use $ x(z) $ as an approximate state of my system. This approximate state should be such that it has a minimal loss of information.

Or variationally,
$$\begin{align}
x = \min_x H(Z | X)
\end{align}$$

In the canonical ensemble this becomes,
$$\begin{align}
H\big(Z\big|X) = - \int dz~ \rho_X\big(x(z)\big)~\rho_Z\big(z\big) \log \rho_Z\big(z\big)
\end{align}$$

## Downsampling Density functions

If we wish to downsample a p.d.f., we can try to minimize the loss of information.

$$\begin{align}
\min H\big(Z\big|X) = - \int dz~ \rho_X\big(x(z)\big)~\rho_Z\big(z\big) \log \rho_Z\big(z\big)
\end{align}$$

As an example, lets consider $ Z \sim \mathcal{N}(0, \mathbb{1}) $,


## Example

Take $ x(z)^2 = A z_1^2 + B z_2^2 $,

$$\begin{align}
\rho_Z(z) d^2z &= Z_z^{-1} e^{-( z_1^2 + z_2^2 )/2} d^2z \\
\end{align}$$

Solving for an intermediate distribution,
$$\begin{align}
\rho(y) &= Z_z^{-1} \int d^2z~\delta\big(y - (A z_1^2 + B z_2^2) \big) e^{-( z_1^2 + z_2^2 )/2} \\
  &= \frac{1}{2\pi} \int_{i\mathbb{R}} \frac{ds}{2\pi} e^{sy} \int d^2z e^{- s(A z_1^2 + B z_2^2)} e^{-( z_1^2 + z_2^2 )/2} \\
  &= \int_{i\mathbb{R}} \frac{ds}{2\pi} e^{sy} \left(1 + \frac{1}{\sqrt{2sA}}\right) \left(1 + \frac{1}{\sqrt{2sB}}\right)
\end{align}$$




